In [260]:
import pandas as pd
import numpy as np

In [1]:
mypath='./new_docs/'
txts='./txts/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['jud_rev_new.txt', 'rom_new.txt', 'luke_8_john_new.txt', 'matt02_new.txt', 'gal_eph_new.txt', 'tit_phl_new.txt', 'matt_new.txt', 'acts_new.txt', 'heb_new.txt', 'thes_tim_new.txt', 'phil_col_new.txt', 'john_new.txt', 'mark01_new.txt', 'jam_jud_new.txt']


In [261]:
#large files
def large_f():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

In [ ]:
#small files
def small_f():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                outfile.write(infile.read())

In [ ]:
#leaking descriptor
def leak_df():
    filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for line in itertools.chain.from_iterable(itertools.imap(open, filnames)):
            outfile.write(line)

In [3]:
t={}
for f in onlyfiles:
    inF=open(mypath+f,"r")
    t[f]=inF.read()


Let's remove punctuation symbols
Punctuation Rules in English
1. the period (or full stop in British English)
2. the comma.
3. the exclamation mark.
4. the question mark.
5. the colon.
6. the semicolon.
7. the quotation mark.
8. the apostrophe.

In [12]:
def stripper(src):
    return src.lower().replace("\n"," ").replace("  "," ").replace("ẹ","e")\
    .replace("!","").replace("?","").replace("“","").replace("ḍ","d")\
    .replace(":","").replace(";","").replace("ị","i").replace("ḅ","b")\
    .replace("'","").replace("”","").replace("ọ","o").replace("ú","u")\
    .replace("'","").replace("”","").replace("ụ","u").replace('"',"")

In [ ]:
for f in t:
    t[f]=stripper(t[f])
    oF=open(txts+f,"a")
    of.write(t[f])